In [ ]:
import ipywidgets as widgets
from IPython.display import display
import functions as functions
import importlib
import warnings
import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError
import os
warnings.filterwarnings('ignore')


# Event Field
## Automated Event File Generation
### by Veering Analytics

#### Please enter dropbox details and select files to upload to kernel

In [ ]:
token_widget = widgets.Textarea(description='DropBox API acess token')
log_widget = widgets.Text(description='Path to logs folder')
config_widget = widgets.Text(description='Path to configs folder')
export_widget = widgets.Text(description='Path to export XML event file')
db_top_pane = widgets.VBox([token_widget, log_widget, config_widget, export_widget])
display(db_top_pane)

In [ ]:
def dropbox_connect(at):
    dropbox.Dropbox(at)

def dropbox_list_files(connect, path):

    dbx = connect

  
    files = dbx.files_list_folder(path).entries
    files_list = []
    for file in files:
        if isinstance(file, dropbox.files.FileMetadata):
            metadata = {
                'name': file.name,
                'path_display': file.path_display,
                'client_modified': file.client_modified,
                'server_modified': file.server_modified
            }
            files_list.append(metadata)

    df = pd.DataFrame.from_records(files_list)
    return df.sort_values(by='server_modified', ascending=False)


class get_dropBox:
    def __init__(self, token,logPath, configPath):
        self.access_token = token
        self.log_path = logPath
        self.config_path = configPath
        self.connect = dropbox.Dropbox(token)

        
    def dropbox_logs(self):
        logs = dropbox_list_files(self.connect, self.log_path)
        return logs
    
    def dropbox_configs(self):
        configs = dropbox_list_files(self.connect, self.config_path)
        return configs

        
class load_files_selector:
    def __init__(self, logsList, configsList):
        self.logs_list = logsList.name
        self.configs_list = configsList.name
        
    def log_selector(self):
        widget = widgets.SelectMultiple(
        options=self.logs_list,
        rows=10,
        description='Logs',
        disabled=False)
        return widget
    
    def configs_selector(self):
        widget = widgets.SelectMultiple(
        options=self.configs_list,
        rows=10,
        description='Logs',
        disabled=False)
        return widget
        

def mid_pane_db():

    dropBox_lists =  get_dropBox(token_widget.value, log_widget.value, config_widget.value)
    file_selectors = load_files_selector(dropBox_lists.dropbox_logs(), dropBox_lists.dropbox_configs())
    log_selector = file_selectors.log_selector()
    config_selector = file_selectors.configs_selector()
    pane = widgets.HBox([log_selector, config_selector])
    mid_pane_db.selected_dict = {'logs': log_selector,
                                'configs': config_selector}
    mid_pane_db.pane = pane
    mid_pane_db.dropBox_lists = dropBox_lists

In [ ]:
db_button = widgets.Button(description='Load DB Files')
db_output = widgets.Output()

@db_output.capture()
def dbload1(b=None):
    mid_pane_db()
    db_output.clear_output()
    with db_output:
        display(mid_pane_db.pane)
    
db_button.on_click(dbload1)
display(db_button)
db_output

In [ ]:
class download_from_db:
    def __init__(self,fileList, filePath, at):
        self.file_list = fileList
        self.file_path = filePath
        self.access_token = at
        
    def dropbox_download_file(self):
        dbx = dropbox.Dropbox(self.access_token)
        
        for file in self.file_list:
            db_path = self.file_path+'/'+file
            local_path = str(os.getcwd())+'/'+file
            print(db_path)
            print(local_path)
            with open(local_path, 'wb') as f:
                metadata, result = dbx.files_download(path=db_path)
                f.write(result.content)
def import_file_from_db(b=None):
    load_logs = download_from_db(mid_pane_db.selected_dict['logs'].value,
                            log_widget.value,
                            token_widget.value)
    load_configs = download_from_db(mid_pane_db.selected_dict['configs'].value,
                            config_widget.value,
                            token_widget.value)
    load_logs.dropbox_download_file()
    load_configs.dropbox_download_file()


In [ ]:
import_button = widgets.Button(description='Import from Dropbox')
import_output = widgets.Output()
   
import_button.on_click(import_file_from_db)
display(import_button)

In [ ]:
def load2(b=None):
    use_config = top_pane_dict['buttons'][0].value
    log_path = top_pane_dict['file_choosers'][0].selected
    config_path = top_pane_dict['file_choosers'][1].selected
    
    try:
        log, var_dict, startDict, raceIndex, marksDict, raceEvents, eventLog, selector_dict_day = functions.load_and_process(log_path,config_path,use_config)
        load2.log = log
        load2.var_dict = var_dict
        load2.startDict = startDict
        load2.raceIndex = raceIndex
        load2.marksDict = marksDict
        load2.raceEvents = raceEvents
        load2.eventLog = eventLog
        load2.selector_dict_day = selector_dict_day
        plots = functions.plot_race(startDict, raceEvents, log, marksDict, raceIndex) # create plots for each race
        plot_tab = functions.build_plot_tabs(plots)
        mid_pane = widgets.VBox([plot_tab,selector_dict_day['formatted']])
        load2.mid_pane = mid_pane
        
    except:
        load2.mid_pane = widgets.Label("Error Loading Files")
    



####  Select log and config files to process

In [ ]:
top_pane_dict = functions.build_top_pane()
display(top_pane_dict['formatted'][0])

In [ ]:
load_button = widgets.Button(description='Load and Process Files')
mid_pane_output = widgets.Output()

    
@mid_pane_output.capture()
def load(change):
    load2()
    mid_pane_output.clear_output()
    with mid_pane_output:
        display(load2.mid_pane)
    
    
load_button.on_click(load)
display(load_button)
mid_pane_output

In [ ]:
def dropbox_upload_file(local_path, local_file, dropbox_file_path,at):
    """Upload a file from the local machine to a path in the Dropbox app directory.

    Args:
        local_path (str): The path to the local file.
        local_file (str): The name of the local file.
        dropbox_file_path (str): The path to the file in the Dropbox app directory.

    Example:
        dropbox_upload_file('.', 'test.csv', '/stuff/test.csv')

    Returns:
        meta: The Dropbox file metadata.
    """

    try:
        dbx = dropbox.Dropbox(at)

        local_file_path = pathlib.Path(local_path) / local_file

        with local_file_path.open("rb") as f:
            meta = dbx.files_upload(f.read(), dropbox_file_path, mode=dropbox.files.WriteMode("overwrite"))

            return meta
    except Exception as e:
        print('Error uploading file to Dropbox: ' + str(e))


In [ ]:
def export2(b=None):
    try:
        selector_dict_day = load2.selector_dict_day
        raceEvents = load2.raceEvents 
        startDict = load2.startDict 
        log = load2.log
        filePath_export = top_pane_dict['file_choosers'][0].selected_path
        date_fileName = str(load2.log.TimeStamp.dropna()[1].date())
        fileName_export = "eventField_"+date_fileName
        events = functions.export_to_xml_worker(selector_dict_day, raceEvents, startDict, log, filePath_export, fileName_export)
        export2.events = events
        
    except:
        print("Failed to export XML file")
        
    try:
        dropbox_upload_file(filePath_export, str(fileName_export)+'.xml', export_widget.value+'/'+str(fileName_export)+'.xml',token_widget.value)
        print("event file uploaded to dropbox")
    
    except:
        print("failed to upload to DropBox")


In [ ]:
export_button = widgets.Button(description='Export Events as XML')
bottom_pane_output = widgets.Output()
   
export_button.on_click(export2)
display(export_button)

